In [9]:
import pandas as pd
import os
import numpy
import datetime
import re
import yfinance as yf

In [10]:
senate_inv = pd.read_csv('/Users/paigeblackstone/Desktop/Portfolio29/Portfolio29/Investment_Management/UpdatedSenatorTrades.csv')
senate_inv2 = pd.read_csv('/Users/paigeblackstone/Desktop/Portfolio29/Portfolio29/Investment_Management/senate_stock_discosures.csv')
senate_inv3 = pd.read_csv('/Users/paigeblackstone/Desktop/Portfolio29/Portfolio29/Investment_Management/SenatorCleaned.csv')

In [11]:
encodings = ['latin1', 'ISO-8859-1', 'cp1252']
for encoding in encodings:
    try:
        congress_inv = pd.read_csv('/Users/paigeblackstone/Desktop/Portfolio29/Portfolio29/Investment_Management/Copy of congress-trading-all (3).csv', encoding=encoding)
        print(f'Successfully read the file with encoding: {encoding}')
        break
    except UnicodeDecodeError:
        print(f'Failed to read the file with encoding: {encoding}')

Successfully read the file with encoding: latin1


In [12]:
congress_inv.columns.to_list()

['Ticker',
 'TickerType',
 'Company',
 'Traded',
 'Transaction',
 'Trade_Size_USD',
 'Status',
 'Subholding',
 'Description',
 'Name',
 'Filed',
 'Party',
 'District',
 'Chamber',
 'Comments',
 'Quiver_Upload_Time',
 'excess_return',
 'State',
 'last_modified']

cleaning up date columns

In [13]:
congress_inv['Traded'] = pd.to_datetime(congress_inv['Traded'], format='%A, %B %d, %Y')

# Display the updated DataFrame and check the data types
print(congress_inv)
print(congress_inv.dtypes)

                       Ticker TickerType  \
0                         NGL         ST   
1                         FCX         ST   
2                           V         ST   
3      APPLE INC. (XNAS:AAPL)         ST   
4                        MSFT         ST   
...                       ...        ...   
46462                     CAT        NaN   
46463                      PG        NaN   
46464                     KSU        NaN   
46465                    EP$C        NaN   
46466                    EP$C        NaN   

                                                 Company     Traded  \
0      NGL ENERGY PARTNERS LP COMMON UNITS REPRESENTI... 2024-03-11   
1                    FREEPORT-MCMORAN, INC. COMMON STOCK 2024-02-29   
2                                              VISA INC. 2024-02-29   
3                              APPLE INC. - COMMON STOCK 2024-02-29   
4                   MICROSOFT CORPORATION - COMMON STOCK 2024-02-29   
...                                          

In [14]:
congress_inv['Filed'] = pd.to_datetime(congress_inv['Filed'], errors='coerce')

# Check for any conversion issues
print(congress_inv['Filed'].head())

0   2024-03-13
1   2024-03-07
2   2024-03-07
3   2024-02-29
4   2024-03-08
Name: Filed, dtype: datetime64[ns]


Cleaning up tickers data and replacing with a mapping of cleaned tickers

In [15]:
ticker_list = congress_inv['Ticker'].unique().tolist()
print(ticker_list)
unique_tickers = ticker_list

['NGL', 'FCX', 'V', 'APPLE INC. (XNAS:AAPL)', 'MSFT', 'BA', 'URI', 'TDG', 'DUK$A', 'ZTS', 'PFE', 'QCOM', 'SBLK', 'ASC', 'SSBK', 'GOGL', 'FLNG', 'LRLCY', 'ARLP', 'BBY', 'SBUX', 'AMD', 'PANW', 'ET', 'ACN', 'NVDA', 'AMAT', 'TGT', 'GLA', 'EQNR', 'PG', 'MMC', 'NWN', 'BDX', 'OKE', 'TEX', 'TXN', 'DBSDY', 'CHRD', 'ILMN', 'HUMA', 'PHG', 'KEY', 'AMZN', 'CTSH', 'UL', 'TSM', 'BKNG', 'MDT', 'REGN', 'MDB', 'META', 'KKR', 'LLY', 'NFLX', 'GLAS FUNDS, LP', 'HAMILTO', 'T$A', 'AFRM', 'SPG', 'LNG', 'TTD', 'DE', 'IBM', 'MRNA', 'FSLR', 'SCHW', 'MS', 'MSCI', 'GOOGL', 'EXAS', 'HTGC', 'COST', 'JPM', 'PRNDY', 'ALNY', 'ISRG', 'VIG', 'USFR', 'GOOG', 'BRK.B', 'LRCX', 'NOW', 'TSLA', 'AAPL', 'ON', 'APH', 'ABG', 'TPH', 'LOW', 'AGIO', 'INTC', 'RPM', 'MEDP', 'LAD', 'MRK', 'SYY', '4.WEEK, MATURE', 'AMGN', 'INTU', 'TJX', 'AON', 'CFG', 'CAT', 'MU', 'CME', 'NKE', 'ABT', 'CRM', 'CB', 'PKG', 'ABBV', 'MA', 'LMT', 'WMT', 'JNJ', 'GIS', 'SCCO', 'MNST', 'CLF', 'IDXX', 'FMAO', 'BABA', 'PAM', 'RTX', 'HQY', 'ADSK', 'DIS', 'NVO', 'CO

In [16]:
cleaned_tickers = []
for ticker in unique_tickers:

    match = re.search(r'\(([^)]+)\)', ticker)
    if match:
        ticker = match.group(1).split(":")[-1]  

    ticker = re.sub(r'[^A-Z0-9-]', '', ticker)

    if ticker:
        cleaned_tickers.append(ticker)

In [17]:
ticker_mapping = dict(zip(congress_inv['Ticker'], cleaned_tickers))
congress_inv['Ticker'] = congress_inv['Ticker'].map(ticker_mapping)
congress_inv = congress_inv[congress_inv['Ticker'].isin(cleaned_tickers)]
print(congress_inv)

      Ticker TickerType                                            Company  \
0        GMT         ST  NGL ENERGY PARTNERS LP COMMON UNITS REPRESENTI...   
1        DDR         ST                FREEPORT-MCMORAN, INC. COMMON STOCK   
2        BCR         ST                                          VISA INC.   
3       AAPL         ST                          APPLE INC. - COMMON STOCK   
4       RYAM         ST               MICROSOFT CORPORATION - COMMON STOCK   
...      ...        ...                                                ...   
46459   MNTA        NaN                        LOCKHEED MARTIN CORPORATION   
46460  VPMAX      Stock        E. I. du Pont de Nemours and Company (NYSE)   
46461   BETR        NaN                           PROCTER & GAMBLE COMPANY   
46462   SIMO        NaN                                  CATERPILLAR, INC.   
46463   BETR        NaN                           PROCTER & GAMBLE COMPANY   

          Traded Transaction       Trade_Size_USD Status  \
0  

observing counts of tickers traded by filed date and by traded date

In [20]:
ticker_counts_by_filed_date = congress_inv.groupby(['Filed', 'Ticker', 'Company', 'Transaction']).size().reset_index(name='Count')
print(ticker_counts_by_filed_date)

ticker_counts_by_traded_date = congress_inv.groupby(['Traded', 'Ticker', 'Company', 'Transaction']).size().reset_index(name='Count')
print(ticker_counts_by_traded_date)

           Filed     Ticker                                          Company  \
0     2014-01-07  36966R5P7                              TJX COMPANIES, INC.   
1     2014-01-07       ADTF                          MASTERCARD INCORPORATED   
2     2014-01-07       ATRO      INTERNATIONAL BUSINESS MACHINES CORPORATION   
3     2014-01-07        BWX                                       3M COMPANY   
4     2014-01-07        BYI                  PHILIP MORRIS INTERNATIONAL INC   
...          ...        ...                                              ...   
23907 2024-03-15        PCH  Southern States Bancshares, Inc. - Common Stock   
23908 2024-03-15       SBLK         Star Bulk Carriers Corp. - Common Shares   
23909 2024-03-15        SPB                                   Starbucks Corp   
23910 2024-03-15       SWAY                        Pfizer, Inc. Common Stock   
23911 2024-03-15         TV             QUALCOMM Incorporated - Common Stock   

       Transaction  Count  
0         P

In [21]:
monthly_counts_by_filed_date = congress_inv.groupby([pd.Grouper(key='Filed', freq='M'), 'Ticker']).size().reset_index(name='Count')
monthly_counts_by_filed_date['Filed'] = monthly_counts_by_filed_date['Filed'].dt.to_period('M').astype(str)

monthly_counts_by_traded_date = congress_inv.groupby([pd.Grouper(key='Traded', freq='M'), 'Ticker']).size().reset_index(name='Count')
monthly_counts_by_traded_date['Filed'] = monthly_counts_by_traded_date['Traded'].dt.to_period('M').astype(str)

collecting historical stock data based on tickers in the congress financial disclosures dataframe, starting in 2022

In [22]:
def get_stock_info(ticker):
    try:
        stock = yf.Ticker(ticker)
        info = stock.info
        if not info:  # Check if info is empty
            raise ValueError("No data found for ticker")
        return {
            'Ticker': ticker,
            'Industry': info.get('industry', 'N/A'),
            'Sector': info.get('sector', 'N/A'),
            'Market Cap': info.get('marketCap', 'N/A'),
            'High Price': info.get('dayHigh', 'N/A'),
            'Low Price': info.get('dayLow', 'N/A'),
            '52 Week High': info.get('fiftyTwoWeekHigh', 'N/A'),
            '52 Week Low': info.get('fiftyTwoWeekLow', 'N/A'),
            'Previous Close': info.get('regularMarketPreviousClose', 'N/A'),
            'PE Ratio': info.get('trailingPE', 'N/A')
        }
    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")
        return None  # Return None if there's an error

def get_monthly_data(ticker, start_date):
    try:
        stock = yf.Ticker(ticker)
        hist = stock.history(start=start_date)
        if hist.empty:  # Check if history data is empty
            raise ValueError("No historical data found for ticker")
        hist['Ticker'] = ticker
        monthly_hist = hist.resample('M').agg({
            'Open': 'mean',
            'High': 'max',
            'Low': 'min',
            'Close': 'mean',
            'Volume': 'sum'
        })
        return monthly_hist
    except Exception as e:
        print(f"Error fetching historical data for {ticker}: {e}")
        return pd.DataFrame()  # Return empty DataFrame if there's an error

# Collect industry information and save per ticker
#for ticker in unique_tickers:
    #stock_info = get_stock_info(ticker)
    #if stock_info:  # Only save if stock_info is not None
        #stock_info_df = pd.DataFrame([stock_info])
        #stock_info_df.to_csv(f'{ticker}_info.csv', index=False)
        #print(f"Industry data for {ticker} saved successfully.")

# Collect monthly data starting from January 1, 2022, and save per ticker
start_date = '2021-01-01'
for ticker in unique_tickers:
    monthly_data = get_monthly_data(ticker, start_date)
    if not monthly_data.empty:
        monthly_data.to_csv(f'{ticker}_monthly_data.csv')
        print(f"Monthly data for {ticker} saved successfully.")

print("Data collection completed.")


Monthly data for NGL saved successfully.
Monthly data for FCX saved successfully.
Monthly data for V saved successfully.


$APPLE INC. (XNAS:AAPL): possibly delisted; No timezone found


Error fetching historical data for APPLE INC. (XNAS:AAPL): No historical data found for ticker
Monthly data for MSFT saved successfully.
Monthly data for BA saved successfully.
Monthly data for URI saved successfully.
Monthly data for TDG saved successfully.


$DUK$A: possibly delisted; No timezone found


Error fetching historical data for DUK$A: No historical data found for ticker
Monthly data for ZTS saved successfully.
Monthly data for PFE saved successfully.
Monthly data for QCOM saved successfully.
Monthly data for SBLK saved successfully.
Monthly data for ASC saved successfully.
Monthly data for SSBK saved successfully.
Monthly data for GOGL saved successfully.
Monthly data for FLNG saved successfully.
Monthly data for LRLCY saved successfully.
Monthly data for ARLP saved successfully.
Monthly data for BBY saved successfully.
Monthly data for SBUX saved successfully.
Monthly data for AMD saved successfully.
Monthly data for PANW saved successfully.
Monthly data for ET saved successfully.
Monthly data for ACN saved successfully.
Monthly data for NVDA saved successfully.
Monthly data for AMAT saved successfully.


...

$WCRX: possibly delisted; No price data found  (1d 2021-01-01 -> 2024-08-09)
Error fetching historical data for WCRX: No historical data found for ticker
$MLPN: possibly delisted; No price data found  (1d 2021-01-01 -> 2024-08-09)
Error fetching historical data for MLPN: No historical data found for ticker
Monthly data for IYG saved successfully.
Monthly data for FEZ saved successfully.
Monthly data for UIS saved successfully.


$GMXRQ: possibly delisted; No timezone found


Error fetching historical data for GMXRQ: No historical data found for ticker


$EP$C: possibly delisted; No timezone found


Error fetching historical data for EP$C: No historical data found for ticker
Data collection completed.
